In [1]:
import pandas as pd

In [2]:
MultiVAE_recon = pd.read_csv('/opt/ml/input/code/output/MultiVAE_recon.csv').to_numpy()
print(MultiVAE_recon)

[[       -inf        -inf        -inf ... -1.94449     0.13540849
  -2.730548  ]
 [-0.69046545  0.35692573  2.4384282  ... -2.2969117  -6.719362
  -3.7252674 ]
 [ 0.13764262 -0.32147062 -0.05545632 ... -3.757159   -4.781424
  -4.8535647 ]
 ...
 [ 1.1122448   0.42727745  1.8475657  ... -2.0716033  -1.3138429
  -4.106259  ]
 [-0.49199325  0.3512059  -0.504378   ... -3.6050904  -6.1360335
  -6.2000923 ]
 [       -inf  0.83166534 -0.01101679 ... -1.8990917  -2.82983
  -3.635313  ]]


In [3]:
RecVAE_recon = pd.read_csv('/opt/ml/input/code/output/RecVAE_recon.csv').to_numpy()
print(RecVAE_recon)

[[       -inf        -inf        -inf ... -1.542791    0.767026
  -3.080206  ]
 [ 0.07644069  0.11381254  2.9982886  ...  0.40977126 -5.4181113
  -3.260325  ]
 [-1.2090167  -1.2670263  -0.26334745 ... -3.2951405  -6.293946
  -5.610749  ]
 ...
 [ 2.0777261   1.3275775   1.9766551  ... -0.7032384  -0.48374152
  -5.1581764 ]
 [-1.9451867   0.7208952  -0.838625   ... -3.5060196  -7.4415174
  -8.823694  ]
 [       -inf  0.9894954   0.5258022  ... -2.7967148  -1.5391778
  -4.0345283 ]]


In [4]:
EASE_recon = pd.read_csv('/opt/ml/input/code/output/EASE_recon.csv').to_numpy()
EASE_recon

array([[ 1.16059412,  0.59100162, -0.06780781, ..., -0.00679623,
        -0.01066291, -0.01034763],
       [ 0.97182629,  0.06162117,  0.10961931, ...,  0.02746037,
        -0.01873638, -0.00940951],
       [ 0.00636936, -0.00426585,  0.01317239, ..., -0.00778428,
        -0.00711079,  0.00552333],
       ...,
       [ 1.01256431,  0.48248603,  0.04911717, ...,  0.00741157,
        -0.02438921,  0.0080125 ],
       [ 0.14896827, -0.05112392,  0.02335233, ..., -0.00838821,
         0.00272924, -0.00682354],
       [ 1.13142675,  0.60900413,  0.06203313, ...,  0.00720961,
         0.00259676,  0.00665914]])

In [5]:
import copy
import numpy as np

RecVAE_recon_norm = copy.deepcopy(RecVAE_recon)

for i in range(len(RecVAE_recon_norm)):
    tmp = [score for score in RecVAE_recon_norm[i] if score != -np.inf and score!=np.inf ]
    if len(RecVAE_recon_norm[i])-len(tmp) <=1000 : weight = 0.8
    else : weight = 0.2
    # for score in RecVAE_recon_norm[i]:
    #     if score != -np.inf and score!=np.inf:
    #         tmp.append()
    tmp_mean = np.mean(tmp, axis=0)
    tmp_std = np.std(tmp, axis=0)
    for j, score in enumerate(RecVAE_recon_norm[i]):
        if score != -np.inf and score!=np.inf:
            RecVAE_recon_norm[i][j] = (RecVAE_recon_norm[i][j] - tmp_mean)/tmp_std
            RecVAE_recon_norm[i][j]*=weight

In [6]:
import copy

MultiVAE_recon_norm = copy.deepcopy(MultiVAE_recon)

for i in range(len(MultiVAE_recon_norm)):
    # user_sum = 0
    # for score in MultiVAE_recon_norm[i]:
    #     if score != -np.inf and score!=np.inf:
    #         user_sum += score

    tmp = [score for score in MultiVAE_recon_norm[i] if score != -np.inf and score!=np.inf ]
    if len(MultiVAE_recon_norm[i])-len(tmp) >1000 : weight = 0.8
    else : weight = 0.2

    tmp_mean = np.mean(tmp, axis=0)
    tmp_std = np.std(tmp, axis=0)

    for j, score in enumerate(MultiVAE_recon_norm[i]):
        if score != -np.inf and score!=np.inf:
            MultiVAE_recon_norm[i][j] = (MultiVAE_recon_norm[i][j] - tmp_mean)/tmp_std
            MultiVAE_recon_norm[i][j] *= weight

KeyboardInterrupt: 

In [ ]:
raw_data = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')

unique_sid = pd.unique(raw_data['item'])
unique_uid = pd.unique(raw_data['user'])
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

id2show = list(show2id.keys())

In [ ]:
import copy
import numpy as np

EASE_show2id = dict((sid, i) for (i, sid) in enumerate(sorted(raw_data['item'].unique())))
EASE_recon_norm = copy.deepcopy(EASE_recon)

for i in range(len(EASE_recon_norm)):
    #tmp = [EASE_score for RecVAE_score, EASE_score in zip(RecVAE_recon_norm[i], EASE_recon_norm[i]) if RecVAE_score != -np.inf and RecVAE_score!=np.inf ]
    tmp = list()
    for idx, RecVAE_score in enumerate(RecVAE_recon_norm[i]):
        if RecVAE_score != -np.inf and RecVAE_score != np.inf
            tmp.append(EASE_recon_norm[i][EASE_show2id[id2show[idx]]])

    # if len(RecVAE_recon_norm[i])-len(tmp) <=1000 : weight = 0.8
    # else : weight = 0.2
    # for score in RecVAE_recon_norm[i]:
    #     if score != -np.inf and score!=np.inf:
    #         tmp.append()
    tmp_mean = np.mean(tmp, axis=0)
    tmp_std = np.std(tmp, axis=0)
    for j, score in enumerate(EASE_recon_norm[i]):
        if RecVAE_recon_norm[i][j] != -np.inf and RecVAE_recon_norm[i][j]!=np.inf:
            #EASE_recon_norm[i][j] = (EASE_recon_norm[i][j] - tmp_mean)/tmp_std
            EASE_recon_norm[i][EASE_show2id[id2show[j]]] = (EASE_recon_norm[i][EASE_show2id[id2show[j]]] - tmp_mean)/tmp_std
            #EASE_recon_norm[i][j]*=weight
        else:
            EASE_recon_norm[i][j] = -np.inf
            EASE_recon[i][j] = -np.inf

In [ ]:
tmp_std

In [64]:
EASE_recon

array([[       -inf,        -inf,        -inf, ..., -0.00679623,
        -0.01066291, -0.01034763],
       [ 0.97182629,  0.06162117,  0.10961931, ...,  0.02746037,
        -0.01873638, -0.00940951],
       [ 0.00636936, -0.00426585,  0.01317239, ..., -0.00778428,
        -0.00711079,  0.00552333],
       ...,
       [ 1.01256431,  0.48248603,  0.04911717, ...,  0.00741157,
        -0.02438921,  0.0080125 ],
       [ 0.14896827, -0.05112392,  0.02335233, ..., -0.00838821,
         0.00272924, -0.00682354],
       [       -inf,  0.60900413,  0.06203313, ...,  0.00720961,
         0.00259676,  0.00665914]])

In [89]:
raw_data = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')

unique_sid = pd.unique(raw_data['item'])
unique_uid = pd.unique(raw_data['user'])
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))


submission_norm_user = list()
submission_norm_item = list()

In [90]:
import bottleneck as bn
import numpy as np

#ensemble_recon_norm = MultiVAE_recon_norm+RecVAE_recon_norm + EASE_recon_norm*0.8

ensemble_recon_norm = EASE_recon

for i in range(len(ensemble_recon_norm)):
    idxes_bn = bn.argpartition(-EASE_recon[i], 10)[:10] # 유저에게 추천할 10개 영화를 가져옴
    idxes = np.argpartition(EASE_recon[i], -10)[-10:]
    tmp = list()
    # id2show 과정
    for j in idxes:                    
        tmp.append(list(show2id.keys())[j]) # id2show # = tmp.append(raw_data['item'].unique()[j])
    submission_norm_item.append(tmp)

In [93]:
set(idxes)==set(idxes_bn)

True

In [92]:
idxes_bn

array([ 226, 2670, 3567, 3399, 2762,  161, 1719, 2344,  659, 2659])

In [75]:
submission_norm_item = np.array(submission_norm_item).reshape(-1, 1)
submission_norm_user = raw_data['user'].unique().repeat(10)
submission_norm_user = np.array(submission_norm_user).reshape(-1, 1)

submission_norm_result = np.hstack((submission_norm_user,submission_norm_item))

submission_norm_result = pd.DataFrame(submission_norm_result, columns=['user','item'])  
submission_norm_result.to_csv('/opt/ml/input/code/output/submission_EASE.csv', index=False)

In [70]:
raw_data = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')
submission_user = list()
submission_item = list()

In [71]:
unique_sid = pd.unique(raw_data['item'])
unique_uid = pd.unique(raw_data['user'])
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [72]:
import bottleneck as bn
import numpy as np

for i in range(len(ensemble_recon)):
    #idxes = bn.argpartition(-ensemble_recon[i], 10)[:10] # 유저에게 추천할 10개 영화를 가져옴
    idesex = np.argpartition(EASE_recon[i], -10)[-10:]
    tmp = list()
    # id2show 과정
    for j in idxes:                    
        tmp.append(list(show2id.keys())[j]) # id2show # = tmp.append(raw_data['item'].unique()[j])
    submission_item.append(tmp)

NameError: name 'ensemble_recon' is not defined

In [16]:
import numpy as np

submission_item = np.array(submission_item).reshape(-1, 1)
submission_user = raw_data['user'].unique().repeat(10)
submission_user = np.array(submission_user).reshape(-1, 1)

submission_result = np.hstack((submission_user,submission_item))

submission_result = pd.DataFrame(submission_result, columns=['user','item'])  
submission_result.to_csv('/opt/ml/input/code/output/submission_ensemble.csv', index=False)

In [13]:
ensemble_recon_norm = RecVAE_recon_norm+MultiVAE_recon_norm
ensemble_recon_norm

array([[       -inf,        -inf,        -inf, ...,  0.14252954,
         2.25595863, -0.98664339],
       [ 1.01883781,  1.47645796,  3.39074727, ...,  0.45664406,
        -3.50305168, -1.4615251 ],
       [ 1.00912631,  0.73428441,  1.29831043, ..., -1.98757644,
        -3.79804858, -3.55235351],
       ...,
       [ 3.02287326,  2.42410858,  3.31759699, ...,  0.51076999,
         0.93877496, -2.1228982 ],
       [ 0.88669586,  2.09256213,  1.21253069, ..., -1.0818904 ,
        -3.48191565, -3.92724124],
       [       -inf,  2.64665847,  1.98396785, ..., -0.65758632,
        -0.49308467, -2.16603424]])

In [38]:
ensemble_recon

array([[        -inf,         -inf,         -inf, ...,  -3.487281  ,
          0.90243449,  -5.810754  ],
       [ -0.61402476,   0.47073827,   5.4367168 , ...,  -1.88714044,
        -12.1374733 ,  -6.9855924 ],
       [ -1.07137408,  -1.58849692,  -0.31880377, ...,  -7.0522995 ,
        -11.07537   , -10.4643137 ],
       ...,
       [  3.1899709 ,   1.75485495,   3.8242208 , ...,  -2.7748417 ,
         -1.79758442,  -9.2644354 ],
       [ -2.43717995,   1.0721011 ,  -1.343003  , ...,  -7.11111   ,
        -13.5775509 , -15.0237863 ],
       [        -inf,   1.82116074,   0.51478541, ...,  -4.6958065 ,
         -4.3690078 ,  -7.6698413 ]])

In [84]:
import bottleneck as bn

top_30_item = list()
top_30_user = list()
top_30_score = list()

for i in range(len(MultiVAE_recon)):
    idxes = bn.argpartition(-MultiVAE_recon[i], 30)[:30] # 유저에게 추천할 10개 영화를 가져옴
    #if i==0 : print(idxes)
    score_id_list = list()
    #score_list = list()
    # id2show 과정
    for j in idxes:                    
        score_id_list.append([MultiVAE_recon[i][j], list(show2id.keys())[j]]) # id2show # = tmp.append(raw_data['item'].unique()[j])
        #score_list.append(RecVAE_recon[i][j])
    score_id_list = sorted(score_id_list, reverse=True)
    top_30_item.append(np.array(score_id_list).T[1])
    top_30_score.append(np.array(score_id_list).T[0])

In [85]:
top_30_item = np.array(top_30_item).reshape(-1, 1)
top_30_score = np.array(top_30_score).reshape(-1, 1)
top_30_user = raw_data['user'].unique().repeat(30)
top_30_user = np.array(top_30_user).reshape(-1, 1)

top_30_result = np.hstack((top_30_user,top_30_item, top_30_score))

top_30_result = pd.DataFrame(top_30_result, columns=['user','item','score'])  
top_30_result.to_csv('/opt/ml/input/code/output/MultiVAE_recon_top_30.csv', index=False)

In [86]:
import bottleneck as bn

top_50_item = list()
top_50_user = list()
top_50_score = list()

for i in range(len(MultiVAE_recon)):
    idxes = bn.argpartition(-MultiVAE_recon[i], 30)[:30] # 유저에게 추천할 10개 영화를 가져옴
    #id_list = list()
    #score_list = list()
    score_id_list = list()
    # id2show 과정
    for j in idxes:                    
        score_id_list.append([MultiVAE_recon[i][j], list(show2id.keys())[j]]) 
        #id_list.append(list(show2id.keys())[j]) # id2show # = tmp.append(raw_data['item'].unique()[j])
        #score_list.append(RecVAE_recon[i][j])
    top_50_item.append(np.array(score_id_list).T[1])
    top_50_score.append(np.array(score_id_list).T[0])

In [87]:
top_50_item = np.array(top_50_item).reshape(-1, 1)
top_50_score = np.array(top_50_score).reshape(-1, 1)
top_50_user = raw_data['user'].unique().repeat(30)
top_50_user = np.array(top_50_user).reshape(-1, 1)

top_50_result = np.hstack((top_50_user,top_50_item, top_50_score))

top_50_result = pd.DataFrame(top_30_result, columns=['user','item','score'])  
top_50_result.to_csv('/opt/ml/input/code/output/MultiVAE_recon_top_50.csv', index=False)